In [1]:
import pandas as pd
from services.jira_service import JiraService  # seu service acima
from datetime import datetime
from services.jira_parser import parse_issues_to_dataframe_espaco_de_projetos, prepare_dataframe_for_json_export
import json


jira = JiraService()
issues = jira.get_all_issues_from_project("EP")  # ou outro projeto

for issue in issues["issues"]:
    status = issue['fields']['status']['name']
    if str(status).lower() == "backlog priorizado":
        print(f"Issue: {issue['key']} - Nome: {issue['fields']['summary']} -Status: {status}")


    # if issue["key"] == "EP-3":
    #     #parser para json
    #     df = parse_issues_to_dataframe_espaco_de_projetos([issue])
    #     df.to_json("ep_3.json", orient="records")
    #     break






🔄 Buscando todas as issues ordenadas por Rank do projeto 'EP'...
✅ 20 issues encontradas no projeto 'EP'.
Issue: EP-6 - Nome: Renovação de Certificados SSL Olhar Certo -Status: Backlog Priorizado
Issue: EP-15 - Nome: Contas a Pagar Simplificado -Status: Backlog Priorizado
Issue: EP-1 - Nome: Criação do Dashboard Geral Marketing -Status: Backlog Priorizado
Issue: EP-5 - Nome: Careflow -Status: Backlog Priorizado


In [ ]:

for i, issue in enumerate(issues["issues"]):
    if issue['fields']['status']['name'] == "Backlog":
        print(f"Título - {issue['fields']['summary']}")

    # if issue["key"] == "EP-3":
    #     print(f"ID - {issue['id']}")
    #     print(f"Tipo - {issue['fields']['issuetype']['name']}")
    #     print(f"Chave - {issue['key']}")
    #     print(f"Título - {issue['fields']['summary']}")
    #     print(f"Prioridade - {issue['fields']['priority']['name']}")
    #     print(f"Descrição - {issue['fields']['description']['content'][0]['content'][0]['text']}")
    #     print(f"Aprovador por (diretor) - {issue['fields']['customfield_10250']}")
    #     print(f"Beneficios Esperados - {issue['fields']['customfield_10248']['content'][0]['content'][0]['text']}")
    #     print(f"Status - {issue['fields']['status']['name']}")
    #     print(f"Grupo Solicitante - {issue['fields']['customfield_10083']['value']}")
    #     print(f"Departamento solicitante - {issue['fields']['customfield_10245']['value']}")
    #     print(f"Solicitante - {issue['fields']['customfield_10093']['content'][0]['content'][0]['text']}")
    #     print(f"Responsável - {issue['fields']['assignee']['displayName']}")
    #     print(f"Telefone do Solicitante - {issue['fields']['customfield_10246']}")
    #     print(f"Email do Solicitante - {issue['fields']['customfield_10247']}")
    #     print(f"Relator - {issue['fields']['creator']['displayName']}")
    #     print(f"Categoria - {issue['fields']['labels'][0]}")
    #     print(f"Estimativa original - {issue['fields']['timeoriginalestimate']}")
    #     print(f"Controle de tempo - {issue['fields']['timetracking']['timeSpent']}")
    #     print(f"Investimento Esperado - {issue['fields']['customfield_10249']}")
    #     print(f"Target start - {issue['fields']['customfield_10022']}")
    #     print(f"Target end - {issue['fields']['customfield_10023']}")
    #     print(f"Data de criação - {issue['fields']['created']}")
    #     print(f"Data de atualização - {issue['fields']['updated']}")
    #     print(f"Data de término - {issue['fields']['resolutiondate']}")


 




